In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
#from models import AlexNet
from torchvision import models
import os

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = '../data/'
test_dir = 'normal_test'
classes = sorted(os.listdir(data_dir + test_dir))
batch_size = 8

In [ ]:
mean = [0.44947562, 0.46524084, 0.40037745]
std = [0.18456618, 0.16353698, 0.20014246]
#mean = [0.14318287, 0.19182085, 0.10939839]
#std = [0.20657195, 0.25984347, 0.1585114]

data_transforms = {
        'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)])}

test_images = datasets.ImageFolder(os.path.join(data_dir, test_dir),
                    data_transforms['test'])

test_dataloader = DataLoader(test_images, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
model = models.AlexNet()
model.classifier[6] = nn.Linear(4096, 10)
model.load_state_dict(torch.load('alexnet_pretrained_erba.model', map_location=str(device)))
model.eval()

In [ ]:
def image_show(tensors):

    num_rows = 1
    num_cols = len(labels)
    fig = plt.figure(figsize=(num_cols, num_rows))
    i = 0
    for t in tensors:
        t = np.transpose(t.cpu().numpy(), (1, 2, 0))
        t = np.asarray(std).mean() * t + np.asarray(mean).mean()
        t = np.clip(t, 0, 1)

        ax1 = fig.add_subplot(num_rows, num_cols, i+1)
        fig.set_size_inches(18.5, 10.5)
        ax1.imshow(t, interpolation='none')
        ax1.axis('off')
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        ax1.set_title('Default')
        i += 1

    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.show()

In [ ]:
def image_show(img):
    img = np.asarray(std).mean() * img + np.asarray(mean).mean()
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.pause(0.001)

In [ ]:
class Buffered:
    def __init__(self):
        self.buffer_normal_image = []
        self.buffer_heatmap_image = []
        self.buffer_label = []        
        
    def show_buffered_img(self, normal, heatmap, label):
        self.buffer_heatmap_image.append(heatmap)
        print(len(self.buffer_heatmap_image))
        self.buffer_label.append(label)
        if len(self.buffer_heatmap_image) > 7:
            fig = plt.figure(figsize=(batch_size, 1))
            for i in range(len(self.buffer_heatmap_image)):
                ax1 = fig.add_subplot(1, batch_size, i+1)
                fig.set_size_inches(18.5, 10.5)
                ax1.imshow(self.buffer_heatmap_image[i], interpolation='none')
                ax1.axis('off')
                ax1.set_xticklabels([])
                ax1.set_yticklabels([])
                ax1.set_title(self.buffer_label[i])
            self.buffer_normal_image = []
            self.buffer_heatmap_image = []
            self.buffer_label = []
            plt.subplots_adjust(wspace=0, hspace=0.1)
            plt.show()

In [ ]:
import cv2
import numpy as np
import torch

class CamExtractor():
    """
        Extracts cam features from the model
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

    def save_gradient(self, grad):
        self.gradients = grad

    def forward_pass_on_convolutions(self, x):
        """
            Does a forward pass on convolutions, hooks the function at given layer
        """
        conv_output = None
        for module_pos, module in self.model.features._modules.items():
            x = module(x)  # Forward
            if int(module_pos) == self.target_layer:
                x.register_hook(self.save_gradient)
                conv_output = x  # Save the convolution output on that layer
        return conv_output, x

    def forward_pass(self, x):
        """
            Does a full forward pass on the model
        """
        # Forward pass on the convolutions
        conv_output, x = self.forward_pass_on_convolutions(x)
        x = x.view(x.size(0), -1)  # Flatten
        # Forward pass on the classifier
        x = self.model.classifier(x)
        return conv_output, x


class GradCam():
    """
        Produces class activation map
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        # Define extractor
        self.extractor = CamExtractor(self.model, target_layer)

    def generate_cam(self, input_image, target_class=None):

        # Full forward pass
        # conv_output is the output of convolutions at specified layer
        # model_output is the final output of the model (1, 1000)
        
        # forward completo, viene salvato il gradiente del target layer,
        # conv_out è l'uscita dal convolution target layer
        # model_output è l'uscita dall'ultimo layer della rete
        conv_output, model_output = self.extractor.forward_pass(input_image)
        if target_class is None:
            target_class = np.argmax(model_output.data.numpy())

        # Target for backprop
        
        # Inizializzazione di one_hot_output come tensore tutto di 0 e in posizione [0][target_class]
        # assegna 1
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
        one_hot_output[0][target_class] = 1

        # Zero grads

        # reinizializzazione del gradiente
        # quando si fa backprogation il gradiente va reinizializzato se no le modifiche che verranno
        # apportate nella backpropagation andrebbero a sommarsi al gradiente calcolato precedentemente durante
        # il forward
        self.model.features.zero_grad()
        self.model.classifier.zero_grad()

        # Backward pass with specified target	
        model_output.backward(gradient=one_hot_output, retain_graph=True)
        # Get hooked gradients
        guided_gradients = self.extractor.gradients.data.numpy()[0]
        # Get convolution outputs
        target = conv_output.data.numpy()[0]

        # Get weights from gradients

        # I gradienti sono 1 x ogni neurone quindi viene eseguita una media
        weights = np.mean(guided_gradients, axis=(1, 2))  # Take averages for each gradient
        # Create empty numpy array for cam
        cam = np.ones(target.shape[1:], dtype=np.float32)
        # Multiply each weight with its conv output and then, sum
        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        cam = cv2.resize(cam, (224, 224))
        cam = np.maximum(cam, 0)
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))  # Normalize between 0-1
        cam = np.uint8(cam * 255)  # Scale between 0-255 to visualize
        return cam

In [ ]:
import os
import copy
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from torchvision import models


def convert_to_grayscale(cv2im):
    """
        Converts 3d image to grayscale

    Args:
        cv2im (numpy arr): RGB image with shape (D,W,H)

    returns:
        grayscale_im (numpy_arr): Grayscale image with shape (1,W,D)
    """
    grayscale_im = np.sum(np.abs(cv2im), axis=0)
    im_max = np.percentile(grayscale_im, 99)
    im_min = np.min(grayscale_im)
    grayscale_im = (np.clip((grayscale_im - im_min) / (im_max - im_min), 0, 1))
    grayscale_im = np.expand_dims(grayscale_im, axis=0)
    return grayscale_im


def save_gradient_images(gradient, file_name):
    """
        Exports the original gradient image

    Args:
        gradient (np arr): Numpy array of the gradient with shape (3, 224, 224)
        file_name (str): File name to be exported
    """
    if not os.path.exists('../results'):
        os.makedirs('../results')
    gradient = gradient - gradient.min()
    gradient /= gradient.max()
    gradient = np.uint8(gradient * 255).transpose(1, 2, 0)
    path_to_file = os.path.join('../results', file_name + '.jpg')
    # Convert RBG to GBR
    gradient = gradient[..., ::-1]
    cv2.imwrite(path_to_file, gradient)


def save_class_activation_on_image(org_img, activation_map, file_name):
    """
        Saves cam activation map and activation map on the original image

    Args:
        org_img (PIL img): Original image
        activation_map (numpy arr): activation map (grayscale) 0-255
        file_name (str): File name of the exported image
    """
    if not os.path.exists('../results'):
        os.makedirs('../results')
    # Grayscale activation map
    path_to_file = os.path.join('../results', file_name+'_Cam_Grayscale.jpg')
    cv2.imwrite(path_to_file, activation_map)
    # Heatmap of activation map
    activation_heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)
    path_to_file = os.path.join('../results', file_name+'_Cam_Heatmap.jpg')
    cv2.imwrite(path_to_file, activation_heatmap)
    # Heatmap on picture
    #org_img = cv2.resize(org_img, (224, 224))
    img_with_heatmap = np.float32(activation_heatmap) + np.float32(org_img)
    img_with_heatmap = img_with_heatmap / np.max(img_with_heatmap)
    path_to_file = os.path.join('../results', file_name+'_Cam_On_Image.jpg')
    cv2.imwrite(path_to_file, np.uint8(255 * img_with_heatmap))

def show_class_activation_on_image(org_img, activation_map, buffered):
    #plt.imshow(activation_map, cmap="gray")
    #plt.show()
    activation_heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)
    #image_show(activation_heatmap)
    #img_with_heatmap = np.float32(activation_heatmap) + np.float32(org_img)
    #img_with_heatmap = img_with_heatmap / np.max(img_with_heatmap)
    #image_show(np.uint8(255 * img_with_heatmap))
    
    buffered.show_buffered_img(org_img, frame_extractor(activation_heatmap), "label")

#def image_show(image):
#    b,g,r = cv2.split(image)
#    frame_rgb = cv2.merge((r,g,b))
#    plt.imshow(frame_rgb)
#    plt.show()

def frame_extractor(image):
    b,g,r = cv2.split(image)
    return cv2.merge((r,g,b))

In [ ]:
file_name_to_export = "prova"
grad_cam = GradCam(model, target_layer=11)
buffered = Buffered()
model.train()
for data in test_dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    for i in range(len(inputs)):
        if(labels[i] == 2):
            original = np.transpose(inputs[i].numpy(),(1,2,0))
            single_input = inputs[i].unsqueeze_(0)
            cam = grad_cam.generate_cam(single_input, labels[i])
            print(classes[labels[i]])
            #image_show(original)
            show_class_activation_on_image(original, cam, buffered)
            #save_class_activation_on_image(np.transpose(inputs[i].numpy(),(1,2,0)), cam, file_name_to_export)
        
        #cam = grad_cam.generate_cam(inputs, labels)
        #save_class_activation_on_image(original_image, cam, file_name_to_export)
